# Detector de Neumonia

In [2]:
# carga de librerias
import tensorflow as tf
print(tf.__version__)
import numpy as np
np.set_printoptions(linewidth=200)
from tensorflow import keras
from keras import models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate
import tensorflow.keras.backend as K

1.14.0


In [3]:
# definicion de path de archivos
base_dir = 'D:/Documentos/deep_learning/trabajos/Neumonia/chestxray_1'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Directory with our training cat/dog pictures
train_health_dir = os.path.join(train_dir, 'health')
train_ill_vir_dir = os.path.join(train_dir, 'ill_vir')
train_ill_bac_dir = os.path.join(train_dir, 'ill_bac')

# Directory with our validation cat/dog pictures
test_health_dir = os.path.join(test_dir, 'health')
test_ill_vir_dir = os.path.join(test_dir, 'ill_vir')
test_ill_bac_dir = os.path.join(test_dir, 'ill_bac')

In [4]:
# muestro nombres de las imagenes
train_health_fnames = os.listdir(train_health_dir)
train_ill_vir_fnames = os.listdir(train_ill_vir_dir)
train_ill_bac_fnames = os.listdir(train_ill_bac_dir)

print(train_health_fnames[:10])
print(train_ill_vir_fnames[:10])
print(train_ill_bac_fnames[:10])

['IM-0115-0001.jpeg', 'IM-0117-0001.jpeg', 'IM-0119-0001.jpeg', 'IM-0122-0001.jpeg', 'IM-0125-0001.jpeg', 'IM-0127-0001.jpeg', 'IM-0128-0001.jpeg', 'IM-0129-0001.jpeg', 'IM-0131-0001.jpeg', 'IM-0133-0001.jpeg']
['person1000_virus_1681.jpeg', 'person1003_virus_1685.jpeg', 'person1004_virus_1686.jpeg', 'person1005_virus_1688.jpeg', 'person1007_virus_1690.jpeg', 'person1008_virus_1691.jpeg', 'person1009_virus_1694.jpeg', 'person100_virus_184.jpeg', 'person1010_virus_1695.jpeg', 'person1015_virus_1701.jpeg']
['person1000_bacteria_2931.jpeg', 'person1001_bacteria_2932.jpeg', 'person1002_bacteria_2933.jpeg', 'person1003_bacteria_2934.jpeg', 'person1004_bacteria_2935.jpeg', 'person1005_bacteria_2936.jpeg', 'person1006_bacteria_2937.jpeg', 'person1007_bacteria_2938.jpeg', 'person1008_bacteria_2939.jpeg', 'person1010_bacteria_2941.jpeg']


In [5]:
# veo la cantidad de datos
print('total train health images :', len(os.listdir(train_health_dir)))
print('total train ill with virus images :', len(os.listdir(train_ill_vir_dir)))
print('total train ill with bacteria images :', len(os.listdir(train_ill_bac_dir)))

print('total test health images :', len(os.listdir(test_health_dir)))
print('total test ill with virus images :', len(os.listdir(test_ill_vir_dir)))
print('total test ill with bacteria images :', len(os.listdir(test_ill_bac_dir)))

total train health images : 1341
total train ill with virus images : 1345
total train ill with bacteria images : 2530
total test health images : 234
total test ill with virus images : 148
total test ill with bacteria images : 242


In [6]:
# especifico dimensión de las imágenes
img_dim = (128, 128)
batch_size = 500
classes_num = 3

In [7]:
# Genero las imágenes --> vienen vectores de 3 clases
# uso el image generator para train
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_dim,
    batch_size=batch_size,
    class_mode='binary'
)
# uso el image generator para test
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_dim,
    batch_size=batch_size,
    class_mode='binary'
)

Found 5216 images belonging to 3 classes.
Found 624 images belonging to 3 classes.


In [8]:
# Modelo 1: NN flattern + FC(relu) + FC(softmax) (3capas)
def densenet(input_shape, n_classes, filters = 32):
    
    #batch norm + relu + conv
    def bn_rl_conv(x,filters,kernel=1,strides=1):
        
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters, kernel, strides=strides,padding = 'same')(x)
        return x
    
    def dense_block(x, repetition):
        
        for _ in range(repetition):
            y = bn_rl_conv(x, 4*filters)
            y = bn_rl_conv(y, filters, 3)
            x = concatenate([y,x])
        return x
        
    def transition_layer(x):
        
        x = bn_rl_conv(x, K.int_shape(x)[-1] //2 )
        x = AvgPool2D(2, strides = 2, padding = 'same')(x)
        return x
    
    input = Input (input_shape)
    x = Conv2D(64, 7, strides = 2, padding = 'same')(input)
    x = MaxPool2D(3, strides = 2, padding = 'same')(x)
    
    for repetition in [6,12,24,16]:
        
        d = dense_block(x, repetition)
        x = transition_layer(d)
    x = GlobalAveragePooling2D()(d)
    output = Dense(n_classes, activation = 'softmax')(x)
    
    model = Model(input, output)
    return model

In [9]:
# creo el modelo 1
input_shape = 128, 128, 3
n_classes = 3
model_dnn = densenet(input_shape,n_classes)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
# compilo el modelo
model_dnn.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [11]:
# resumen del modelo
model_dnn.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 32, 32, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 64)   256         max_pooling2d[0][0]              
______________________________________________________________________________________________

In [12]:
# entreno el modelo
history_dnn = model_dnn.fit_generator(
    train_generator,
    steps_per_epoch=50,
    epochs=5,
    validation_data=test_generator,
    validation_steps=50,
    verbose=1
)

Epoch 1/5
50/50 [==============================] - 12847s 257s/step - loss: 1.0887 - acc: 0.6484 - val_loss: 2.6514 - val_acc: 0.3878
Epoch 2/5
50/50 [==============================] - 12073s 241s/step - loss: 0.6039 - acc: 0.7406 - val_loss: 4.4593 - val_acc: 0.3878
Epoch 3/5
50/50 [==============================] - 12166s 243s/step - loss: 0.6018 - acc: 0.7512 - val_loss: 4.5222 - val_acc: 0.3878
Epoch 4/5
 5/50 [==>...........................] - ETA: 2:42:38 - loss: 0.5967 - acc: 0.7428

KeyboardInterrupt: 